## Asynchronous LLM calls

*[Coding along with the Udemy Course [Advanced Retrieval Augmented Generation ](https://www.udemy.com/course/advanced-retrieval-augmented-generation/) by Rémi Connesson]*

Calling the OpenAI API concurrently instead of sequentially.

In [1]:
import pandas as pd
from openai import AsyncOpenAI

In [2]:
api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

Don't be a fool and sent your api key to github


In [3]:
client = AsyncOpenAI(api_key=api_key)

In [4]:
def _msg(role, content):
    return {'role': role, 'content': content}

def system(content):
    return _msg('system', content)

def user(content):
    return _msg('user', content)

def assistant(content):
    return _msg('assistant', content)

In [5]:
chat_history = [
    system("You are a helpful assistant."),
    user("Hey, what is Python?"),
    assistant("Python is a good programming language"),
    user("Why so....???"),
]

In [6]:
from pprint import pprint

In [7]:
pprint(chat_history)

[{'content': 'You are a helpful assistant.', 'role': 'system'},
 {'content': 'Hey, what is Python?', 'role': 'user'},
 {'content': 'Python is a good programming language', 'role': 'assistant'},
 {'content': 'Why so....???', 'role': 'user'}]


In [12]:
completion = await client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = chat_history,
    max_tokens = 50 # limit the output to safe costs
) # returns a coroutine that's an awaitable so we need to put an await in front
# <coroutine object AsyncCompletions.create at 0x1039706c0>

In [13]:
completion # returns a coroutine that's an awaitable so we need to put an await in front

ChatCompletion(id='chatcmpl-AKkP2UK3qIi3HP9gW4gUkMebmTeWo', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Python is considered a good programming language for several reasons:\n\n1. **Readability**: Python syntax is clear and concise, which makes it easier for beginners to learn and understand. The use of indentation to define blocks of code contributes to its readability.\n\n', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729508296, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_482c22a7bc', usage=CompletionUsage(completion_tokens=50, prompt_tokens=41, total_tokens=91, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))

In [16]:
# making completions work concurrently with Python's asyncio library
import asyncio

In [14]:
# making completions work concurrently
completion_task_1 = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = chat_history,
    max_tokens = 50 # limit the output to safe costs
)

completion_task_2 = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = chat_history,
    max_tokens = 50 # limit the output to safe costs
)

In [15]:
completion_task_1, completion_task_2 # two coroutines

(<coroutine object AsyncCompletions.create at 0x12612d6a0>,
 <coroutine object AsyncCompletions.create at 0x12612d8e0>)

In [17]:
# execution calls concurrently with asyncio's gather function
results = await asyncio.gather(completion_task_1, completion_task_2)

In [18]:
results

[ChatCompletion(id='chatcmpl-AKkeAlEzErX7w9TGIMgzNH1F6AEb8', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Python is considered a good programming language for several reasons:\n\n1. **Readability**: Python has a clear and easy-to-read syntax, which makes it a great choice for beginners and allows experienced programmers to understand code quickly.\n\n2. **Versatility', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729509234, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_482c22a7bc', usage=CompletionUsage(completion_tokens=50, prompt_tokens=41, total_tokens=91, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))),
 ChatCompletion(id='chatcmpl-AKkeAI1nCCNe9yYP0hLO7ttwh71Nv', choices=[Choice(finish_reason='length', in

In [24]:
# now let's just create a list of tasks
completion_tasks = []

for i in range(50):
    completion_task = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = chat_history,
        max_tokens = 50 # limit the output to safe costs
    )

    completion_tasks.append(completion_task)

In [25]:
len(completion_tasks) # list of coroutines

50

In [26]:
results_list = await asyncio.gather(*completion_tasks) # unpack them all with *

In [27]:
results_list

[ChatCompletion(id='chatcmpl-AKkoDWdol6BieVstO7RjL0Nsi3J0t', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Python is considered a great programming language for several reasons:\n\n1. **Simplicity and Readability**: Python has a clear and easy-to-understand syntax, which makes it a great choice for beginners. Its design philosophy emphasizes readability, allowing programmers', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729509857, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_482c22a7bc', usage=CompletionUsage(completion_tokens=50, prompt_tokens=41, total_tokens=91, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))),
 ChatCompletion(id='chatcmpl-AKkoDTwMMHlqF9pSaNWL1IUK4mOzx', choices=[Choice(finish_reason='leng

In [28]:
results_list[-1] # having a look at the last one

ChatCompletion(id='chatcmpl-AKkoDj8t4LpPLpzSl288vzIsi1bZh', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Python is considered a good programming language for several reasons:\n\n1. **Ease of Learning**: Python has a simple and readable syntax that makes it accessible for beginners. Its code is often clear and straightforward, which helps new programmers grasp concepts quickly.\n\n2', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729509857, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_e2bde53e6e', usage=CompletionUsage(completion_tokens=50, prompt_tokens=41, total_tokens=91, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))